# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 27 2022 8:55AM,253794,10,0086270316,ISDIN Corporation,Released
1,Dec 27 2022 8:55AM,253794,10,0086270324,ISDIN Corporation,Released
2,Dec 27 2022 8:55AM,253794,10,0086270329,ISDIN Corporation,Released
3,Dec 27 2022 8:55AM,253794,10,0086270330,ISDIN Corporation,Released
4,Dec 27 2022 8:55AM,253794,10,0086270331,ISDIN Corporation,Released
5,Dec 27 2022 8:55AM,253794,10,0086270337,ISDIN Corporation,Released
6,Dec 27 2022 8:55AM,253794,10,0086270342,ISDIN Corporation,Released
7,Dec 27 2022 8:55AM,253794,10,0086270364,ISDIN Corporation,Released
8,Dec 27 2022 8:55AM,253794,10,0086270365,ISDIN Corporation,Released
9,Dec 27 2022 8:55AM,253794,10,0086270366,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
8,253790,Released,5
9,253791,Released,1
10,253792,Released,1
11,253793,Executing,1
12,253794,Released,18


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
253790,NaN,5.0
253791,NaN,1.0
253792,NaN,1.0
253793,1.0,NaN
253794,NaN,18.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
253781,0.0,2.0
253783,0.0,1.0
253784,6.0,23.0
253785,0.0,1.0
253786,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
253781,0,2
253783,0,1
253784,6,23
253785,0,1
253786,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,253781,0,2
1,253783,0,1
2,253784,6,23
3,253785,0,1
4,253786,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,253781,,2
1,253783,,1
2,253784,6,23
3,253785,,1
4,253786,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 27 2022 8:55AM,253794,10,ISDIN Corporation
18,Dec 27 2022 8:51AM,253793,10,Livs Products
19,Dec 27 2022 8:49AM,253792,19,"Graystone, LLC"
20,Dec 27 2022 8:49AM,253791,10,Bio-PRF
21,Dec 27 2022 8:49AM,253790,10,ISDIN Corporation
26,Dec 27 2022 8:38AM,253789,10,"CLINUVEL, Inc."
27,Dec 27 2022 8:37AM,253788,10,"CLINUVEL, Inc."
28,Dec 27 2022 8:10AM,253786,19,"GUSA Granules USA, Inc."
29,Dec 27 2022 8:08AM,253785,19,"GUSA Granules USA, Inc."
30,Dec 27 2022 8:04AM,253784,20,"Exact-Rx, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Dec 27 2022 8:55AM,253794,10,ISDIN Corporation,,18
1,Dec 27 2022 8:51AM,253793,10,Livs Products,1,
2,Dec 27 2022 8:49AM,253792,19,"Graystone, LLC",,1
3,Dec 27 2022 8:49AM,253791,10,Bio-PRF,,1
4,Dec 27 2022 8:49AM,253790,10,ISDIN Corporation,,5
5,Dec 27 2022 8:38AM,253789,10,"CLINUVEL, Inc.",,1
6,Dec 27 2022 8:37AM,253788,10,"CLINUVEL, Inc.",,1
7,Dec 27 2022 8:10AM,253786,19,"GUSA Granules USA, Inc.",,1
8,Dec 27 2022 8:08AM,253785,19,"GUSA Granules USA, Inc.",,1
9,Dec 27 2022 8:04AM,253784,20,"Exact-Rx, Inc.",6,23


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 27 2022 8:55AM,253794,10,ISDIN Corporation,18,
1,Dec 27 2022 8:51AM,253793,10,Livs Products,,1
2,Dec 27 2022 8:49AM,253792,19,"Graystone, LLC",1,
3,Dec 27 2022 8:49AM,253791,10,Bio-PRF,1,
4,Dec 27 2022 8:49AM,253790,10,ISDIN Corporation,5,
5,Dec 27 2022 8:38AM,253789,10,"CLINUVEL, Inc.",1,
6,Dec 27 2022 8:37AM,253788,10,"CLINUVEL, Inc.",1,
7,Dec 27 2022 8:10AM,253786,19,"GUSA Granules USA, Inc.",1,
8,Dec 27 2022 8:08AM,253785,19,"GUSA Granules USA, Inc.",1,
9,Dec 27 2022 8:04AM,253784,20,"Exact-Rx, Inc.",23,6


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 27 2022 8:55AM,253794,10,ISDIN Corporation,18,
1,Dec 27 2022 8:51AM,253793,10,Livs Products,,1
2,Dec 27 2022 8:49AM,253792,19,"Graystone, LLC",1,
3,Dec 27 2022 8:49AM,253791,10,Bio-PRF,1,
4,Dec 27 2022 8:49AM,253790,10,ISDIN Corporation,5,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Dec 27 2022 8:55AM,253794,10,ISDIN Corporation,18.0,NaN
1,Dec 27 2022 8:51AM,253793,10,Livs Products,NaN,1.0
2,Dec 27 2022 8:49AM,253792,19,"Graystone, LLC",1.0,NaN
3,Dec 27 2022 8:49AM,253791,10,Bio-PRF,1.0,NaN
4,Dec 27 2022 8:49AM,253790,10,ISDIN Corporation,5.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 27 2022 8:55AM,253794,10,ISDIN Corporation,18.0,0.0
1,Dec 27 2022 8:51AM,253793,10,Livs Products,0.0,1.0
2,Dec 27 2022 8:49AM,253792,19,"Graystone, LLC",1.0,0.0
3,Dec 27 2022 8:49AM,253791,10,Bio-PRF,1.0,0.0
4,Dec 27 2022 8:49AM,253790,10,ISDIN Corporation,5.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,1522745,26.0,1.0
19,1268927,6.0,0.0
20,253784,23.0,6.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,1522745,26.0,1.0
1,19,1268927,6.0,0.0
2,20,253784,23.0,6.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,26.0,1.0
1,19,6.0,0.0
2,20,23.0,6.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,26.0
1,19,Released,6.0
2,20,Released,23.0
3,10,Executing,1.0
4,19,Executing,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,19,20
Status,,,
Executing,1.0,0.0,6.0
Released,26.0,6.0,23.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,19,20
0,Executing,1.0,0.0,6.0
1,Released,26.0,6.0,23.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,19,20
0,Executing,1.0,0.0,6.0
1,Released,26.0,6.0,23.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()